# Open a microscopy data in any format 

The installation of the package aicsimageio is required. Documentation can be found [here](https://allencellmodeling.github.io/aicsimageio/index.html).  <br>
Steps to follow for the installation of the required package: <br>
> \$ conda activate \<env\> <br>
    \$ pip install aicsimageio <br>
    \$ pip install aicsimageio[base-imageio,nd2,bfio,all] <br>
    \$ pip install aicsimageio readlif>=0.6.4 <br>
    \$ pip install aicsimageio aicspylibczi>=3.0.5 fsspec>=2022.7.1 <br>
    \$ conda install -c conda-forge bioformats_jar <br>

In [2]:
# EXAMPLE
from scyjava import config
from aicsimageio import AICSImage
img = AICSImage("Y:/April05/FluoCells1_BPAE_F36924/FluoCells1_F3624_001/Data32Bit/PSFCZT_Ab_ch00.tif")